In [11]:
# notebooks/02_baseline_failure_scenarios.ipynb

import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json

from src.utils.db import SupplyChainDB

# Setup
db = SupplyChainDB()

with open('data/processed/baseline_metrics.json', 'r') as f:
    baseline = json.load(f)

print("="*70)
print("BASELINE SYSTEM FAILURE SCENARIOS")
print("="*70)
print("\nSimple Reorder Point Logic:")
print("  IF inventory <= reorder_point AND no_pending_orders THEN:")
print("      order_quantity = base_demand * supply_days_target * 1.2")
print("      place_order(quantity)")
print("\nProblems: No forecasting, no event awareness, no optimization")
print("="*70)

# %% [markdown]
# # SCENARIO 1: Black Friday Disaster
# 
# **Problem**: Baseline system doesn't anticipate demand spikes
# 
# Black Friday demand is 2.36x normal, but baseline orders normal quantities

# %% Analyze Black Friday period
black_friday_2023 = '2023-11-24'
black_friday_2024 = '2024-11-29'

# Focus on 2023 (we have complete data)
bf_start = pd.to_datetime(black_friday_2023)
bf_end = bf_start + pd.Timedelta(days=4)

# Get sales during Black Friday
bf_sales = db.query(f"""
    SELECT 
        date,
        product_id,
        SUM(units_sold) as units,
        SUM(revenue) as revenue
    FROM sales
    WHERE date >= '{bf_start.date()}' AND date <= '{bf_end.date()}'
    GROUP BY date, product_id
""")

# Get baseline (2 weeks before)
baseline_start = bf_start - pd.Timedelta(days=14)
baseline_end = bf_start - pd.Timedelta(days=1)

baseline_sales = db.query(f"""
    SELECT 
        product_id,
        AVG(units) as avg_units_per_day,
        AVG(revenue) as avg_revenue_per_day
    FROM (
        SELECT 
            date,
            product_id,
            SUM(units_sold) as units,
            SUM(revenue) as revenue
        FROM sales
        WHERE date >= '{baseline_start.date()}' AND date <= '{baseline_end.date()}'
        GROUP BY date, product_id
    )
    GROUP BY product_id
""")

# Compare actual BF demand vs baseline
bf_by_product = bf_sales.groupby('product_id').agg({
    'units': 'sum',
    'revenue': 'sum'
}).reset_index()

bf_comparison = bf_by_product.merge(baseline_sales, on='product_id', suffixes=('_bf', '_baseline'))
bf_comparison['demand_multiplier'] = bf_comparison['units_bf'] / (bf_comparison['avg_units_per_day'] * 4)  # 4 days of BF
bf_comparison = bf_comparison.sort_values('demand_multiplier', ascending=False)

print("\n🔥 BLACK FRIDAY 2023 - Top 10 Demand Spikes:")
print(bf_comparison[['product_id', 'demand_multiplier', 'revenue_bf']].head(10).to_string(index=False))

# Get inventory levels during Black Friday
bf_inventory = db.query(f"""
    SELECT 
        date,
        product_id,
        SUM(units_on_hand) as total_inventory,
        SUM(stockout) as stockouts
    FROM inventory_snapshots
    WHERE date >= '{bf_start.date()}' AND date <= '{bf_end.date()}'
    GROUP BY date, product_id
""")

# Find products that stocked out during BF
bf_stockouts = bf_inventory[bf_inventory['stockouts'] > 0]['product_id'].unique()

# Calculate lost revenue (products that stocked out during BF spike)
bf_lost = bf_comparison[bf_comparison['product_id'].isin(bf_stockouts)].copy()

# Estimate lost sales: assume we could have sold baseline_multiplier * baseline if we had stock
bf_lost['potential_additional_revenue'] = (
    bf_lost['avg_revenue_per_day'] * 4 * bf_lost['demand_multiplier'] - bf_lost['revenue_bf']
)

total_bf_lost = bf_lost['potential_additional_revenue'].sum()

print(f"\n💰 BLACK FRIDAY LOST REVENUE:")
print(f"   Products that stocked out: {len(bf_stockouts)}")
print(f"   Estimated lost revenue: ${total_bf_lost:,.2f}")

# Visualize
top_bf_lost = bf_lost.nlargest(10, 'potential_additional_revenue')

fig = go.Figure()

fig.add_trace(go.Bar(
    x=top_bf_lost['product_id'],
    y=top_bf_lost['revenue_bf'],
    name='Actual Revenue',
    marker_color='lightblue'
))

fig.add_trace(go.Bar(
    x=top_bf_lost['product_id'],
    y=top_bf_lost['potential_additional_revenue'],
    name='Lost Revenue (Stockout)',
    marker_color='red'
))

fig.update_layout(
    title='Black Friday 2023: Actual vs Lost Revenue',
    xaxis_title='Product',
    yaxis_title='Revenue ($)',
    barmode='stack',
    height=500
)

fig.show()

# SCENARIO 1 SUMMARY
scenario_1 = {
    'name': 'Black Friday Disaster',
    'lost_revenue': float(total_bf_lost),
    'products_affected': len(bf_stockouts),
    'avg_demand_spike': float(bf_comparison['demand_multiplier'].mean()),
    'description': 'Baseline system orders normal quantities, misses 2.36x demand spike'
}

print(f"\n✅ SCENARIO 1: ${scenario_1['lost_revenue']:,.2f} lost")

✅ Connected to database: data/processed/supply_chain.duckdb
BASELINE SYSTEM FAILURE SCENARIOS

Simple Reorder Point Logic:
  IF inventory <= reorder_point AND no_pending_orders THEN:
      order_quantity = base_demand * supply_days_target * 1.2
      place_order(quantity)

Problems: No forecasting, no event awareness, no optimization


KeyError: 'units_bf'